In [1]:
from qiskit_emulator import DellHybridProvider

In [2]:
provider = DellHybridProvider()

In [3]:
provider.remote('http://qre-vf.oro-sandbox-small1.k8s.cec.lab.emc.com')

In [4]:
provider.runtime.programs()

{}

In [5]:
provider.runtime.pprint_programs()

In [6]:
RUNTIME_PROGRAM = """
from qiskit.compiler import transpile, schedule


def main(
    backend,
    user_messenger,
    circuits,
    **kwargs,
):

    user_messenger.publish({'results': 'intermittently'})

    circuits = transpile(
        circuits,
    )

    if not isinstance(circuits, list):
        circuits = [circuits]

    # Compute raw results
    result = backend.run(circuits, **kwargs).result()

    user_messenger.publish({'results': 'finally'})
    user_messenger.publish(result.to_dict(), final=True)
    print("job complete successfully")
"""

In [7]:
RUNTIME_PROGRAM_METADATA = {
    "max_execution_time": 600,
    "description": "Qiskit test program"
}

In [16]:
program_id = provider.runtime.upload_program(RUNTIME_PROGRAM, metadata=RUNTIME_PROGRAM_METADATA)

In [17]:
proglist = provider.runtime.programs()
proglist

{'6a472dcbf845': <qiskit.providers.ibmq.runtime.runtime_program.RuntimeProgram at 0x7f73ac1a7ac0>}

In [18]:
program_id

'a96ef34a4f10'

In [19]:
provider.runtime.pprint_programs(refresh=True)

6a472dcbf845:
  Name: 6a472dcbf845
  Description: Qiskit test program
  Version: 0
  Creation date: 
  Max execution time: 600
  Input parameters:
    none
  Interim results:
    none
  Returns:
    none


In [11]:
from qiskit import QuantumCircuit, execute, transpile
qc = QuantumCircuit(2, 2)
qc.h(0)
qc.cx(0, 1)
qc.measure([0, 1], [0, 1])

program_inputs = {
    'circuits': qc,
}

In [12]:
job = provider.runtime.run(program_id, options=None, inputs=program_inputs)

In [13]:
result = job.result(timeout=120)

In [15]:
job.status()

'Completed'